In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNetCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib

#Jets adding
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV

In [21]:
data = pd.read_csv("C:\\Users\\grovenbm\\Documents\\oms_analytics\\CSE 6242\\Project\\Tests\\philly_final_dataset_Reduced.csv")


In [23]:
y = data['Est TOM in Days']
data_Features = data.drop(columns=['Est TOM in Days','avg_days_on_market'])
data_hot = data_Features
data_hot = pd.get_dummies(data_hot, drop_first=True)


In [24]:
#train test split
#X_train, X_test, y_train, y_test = train_test_split(data10_hot, y, test_size=0.3, random_state=1)
X_train, X_test, y_train, y_test = train_test_split(data_hot, y, test_size=0.3, random_state=1)

# Standardize full feature set for Elastic Net variable selection
scaler_full = StandardScaler()
X_train_scaled_full = scaler_full.fit_transform(X_train)
X_test_scaled_full = scaler_full.transform(X_test)

In [25]:
# Variable selection with Elastic Net
elasticNet = ElasticNetCV(l1_ratio=0.5, cv=5)
elasticNet.fit(X_train_scaled_full, y_train)

# Get the selected features
coefficients = elasticNet.coef_
selected_features = data_hot.columns[coefficients != 0]
print("Selected features:")
print(selected_features)

Selected features:
Index(['zip_code', 'central_air', 'garage_spaces', 'number_of_bedrooms',
       'number_of_bathrooms', 'number_stories', 'total_area', 'year_built',
       'has_basements', 'sale_price', 'active_inventory', 'new_listings',
       'pending_sales', 'homes_sold', 'sale_to_list_ratio',
       'percent_above_list', 'off_market_2w', 'avg_list_price_per_sqft',
       'avg_listed_price', 'avg_price_per_sqft', 'avg_price_sold'],
      dtype='object')


In [26]:
print("--------------------------------------End of Variable Selection--------------------------------------")

--------------------------------------End of Variable Selection--------------------------------------


In [27]:
#Instiantiate model
LMmodel = LinearRegression()

#fit model to training data. Using only selected features
X_train_selected = X_train[selected_features]

LMmodel.fit(X_train_selected, y_train)

#predict on test data
x_test_selected = X_test[selected_features]
y_pred = LMmodel.predict(x_test_selected)

print("Predictions:")
print(y_pred)

#calculate RMSE and R2
RMSE = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE:")
print(RMSE)
print("R2:")
R2 = r2_score(y_test, y_pred)
print(R2)

#Access coefficients
coefficients = LMmodel.coef_
print("Coefficients:")
print(coefficients)

#Access intercept
intercept = LMmodel.intercept_
print("Intercept:")
print(intercept)

Predictions:
[53.9093813  58.76900196 84.39134968 ... 90.25603444 71.80279566
 58.11903003]
RMSE:
18.18708475931443
R2:
0.6080361045559948
Coefficients:
[ 9.41091558e-02 -3.34858111e+00 -4.45267155e+00  2.27282301e-01
  1.78376027e-01  1.25786965e+00 -1.88395056e-07 -7.88117414e-03
 -1.46508316e+00 -2.06519092e-07  2.55078783e-02 -4.14170884e-01
  1.82207274e+00 -1.61422739e+00  2.06329177e+02 -3.42176524e+01
 -6.65862603e+01  6.41376087e-01 -1.53061081e-04 -6.56077761e-01
  1.89371852e-04]
Intercept:
-1895.7381970064152


In [28]:
#Instantiating Random Forest Model
RFmodel = RandomForestRegressor(n_estimators=50, random_state=1,n_jobs=-1 ,verbose=1)

#RF model fit
RFmodel.fit(X_train, y_train)

#Sanity check
print("Progress...")

#Predicting on test data
y_pred_RF = RFmodel.predict(X_test)
print("Predictions:")
print(y_pred_RF)

# Calculate RMSE and R2
RMSE = np.sqrt(mean_squared_error(y_test, y_pred_RF))
print("RMSE:")
print(RMSE)
print("R2:")
R2 = r2_score(y_test, y_pred_RF)
print(R2)

# Get feature names
feature_names = X_train.columns

# Access feature importances (since Random Forest models provide feature importance)
print("Feature Importances:")
feature_importance = pd.Series(RFmodel.feature_importances_, index=feature_names)
feature_importance = feature_importance.sort_values(ascending=False)
print(feature_importance)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.


Progress...
Predictions:
[47. 49. 77. ... 75. 44. 51.]
RMSE:
0.5072968887479335
R2:
0.9996950390153779
Feature Importances:
homes_sold                 0.433863
pending_sales              0.256126
off_market_2w              0.096378
total_area                 0.075603
avg_price_per_sqft         0.036229
new_listings               0.025956
active_inventory           0.024417
sale_to_list_ratio         0.018355
zip_code                   0.016648
percent_above_list         0.004721
avg_list_price_per_sqft    0.004582
avg_price_sold             0.003845
avg_listed_price           0.002197
total_livable_area         0.000352
sale_price                 0.000205
garage_spaces              0.000188
number_of_bedrooms         0.000090
number_stories             0.000087
number_of_bathrooms        0.000074
year_built                 0.000062
has_basements              0.000020
central_air                0.000001
crimes_per_100k            0.000000
dtype: float64


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    0.4s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed:    0.0s finished


In [29]:
# Instantiate Gradient Boosting model

'''
GBmodel = GradientBoostingRegressor(
    n_estimators=200,
    learning_rate=0.15,
    max_depth=4,
    random_state=1,
    verbose=1
)
'''
GBmodel = GradientBoostingRegressor(
    subsample= 1.0, n_estimators= 500,
    min_samples_split= 2, min_samples_leaf= 1,
    max_features= None, max_depth= 4, learning_rate= 0.15
)

# Fit the model on untransformed target
GBmodel.fit(X_train, y_train)

# Predict on test data
y_pred_GB = GBmodel.predict(X_test)

# Evaluate performance
RMSE_GB = np.sqrt(mean_squared_error(y_test, y_pred_GB))
R2_GB = r2_score(y_test, y_pred_GB)

print("Predictions:")
print(y_pred_GB)
print("RMSE (Gradient Boosting):", RMSE_GB)
print("R² (Gradient Boosting):", R2_GB)

# Get feature names
feature_names = X_train.columns

# Access feature importances
print("Feature Importances:")
feature_importance = pd.Series(GBmodel.feature_importances_, index=feature_names)
feature_importance = feature_importance.sort_values(ascending=False)
print(feature_importance)

Predictions:
[47.00040987 48.99981465 76.9989088  ... 75.0150646  43.99524791
 50.99968018]
RMSE (Gradient Boosting): 0.15558822609490097
R² (Gradient Boosting): 0.9999713137788496
Feature Importances:
homes_sold                 4.171028e-01
pending_sales              2.660415e-01
off_market_2w              1.113432e-01
total_area                 5.162176e-02
active_inventory           4.107912e-02
avg_price_per_sqft         3.429181e-02
sale_to_list_ratio         1.792192e-02
percent_above_list         1.141453e-02
avg_price_sold             1.101152e-02
zip_code                   1.013762e-02
avg_listed_price           9.477905e-03
avg_list_price_per_sqft    8.844905e-03
new_listings               7.590357e-03
total_livable_area         1.052193e-03
garage_spaces              7.377014e-04
year_built                 1.996220e-04
sale_price                 1.105198e-04
number_stories             1.976533e-05
number_of_bedrooms         8.065449e-07
number_of_bathrooms        4.428279e-0

In [ ]:
# Define parameter distribution
param_dist = {
    'n_estimators': [100, 200, 300, 500],
    'learning_rate': [0.01, 0.05, 0.1, 0.15],
    'max_depth': [3, 4, 5, 6],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 3, 5],
    'subsample': [0.8, 1.0],
    'max_features': ['sqrt', 'log2', None]
}

# Instantiate base model
gbr = GradientBoostingRegressor(random_state=1)

# Set up randomized search
rand_search = RandomizedSearchCV(
    estimator=gbr,
    param_distributions=param_dist,
    n_iter=50,               # Number of random combinations to try
    scoring='r2',            # Optimize for R² score
    cv=3,                    # 3-fold cross-validation
    verbose=2,
    n_jobs=-1,               # Use all available cores
    random_state=42
)

# Fit to training data
rand_search.fit(X_train, y_train)

# Print best parameters and score
print("Best Parameters Found:")
print(rand_search.best_params_)
print("Best Cross-Validated R² Score:")
print(rand_search.best_score_)

# Evaluate on test set
best_gbr = rand_search.best_estimator_
y_pred_best = best_gbr.predict(X_test)

rmse_best = np.sqrt(mean_squared_error(y_test, y_pred_best))
r2_best = r2_score(y_test, y_pred_best)

print("\nTest Set Evaluation")
print("RMSE:", rmse_best)
print("R²:", r2_best)

#with open("modelTOM.pkl", "wb") as f:
    #import pickle
joblib.dump(best_gbr, 'modelTOM.pkl')

Fitting 3 folds for each of 50 candidates, totalling 150 fits
